In [1]:
import pandas as pd
import datetime
import pymongo
import json
from bson import Binary, Code, json_util, ObjectId
from bson.json_util import dumps, RELAXED_JSON_OPTIONS
import pygeoj
import os
import itertools 
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Create a Mongo Connection

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.education_data

# Scrape graduation rate data

In [3]:
# files
rootdir = './data/4YearGraduation'

# lists defined
districts_list = []
years = ['2018','2015','2014','2016','2017']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        districts_list.append(data)
#         districts_list.sort()
districts_list.pop(0)
# districts_list.reverse()
grad_df=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(districts_list, years):
    data = file
#     defined variables labels
    grad_label = "grad_"+ year
    df_label = grad_label +"_df"
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)

#     replace * with 0
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('*','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('*','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('*','0')
    df_label['FOUR_YR_GRAD_RATE'] = df_label['FOUR_YR_GRAD_RATE'].str.replace('-','0')
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = df_label['FOUR_YR_ADJ_COHORT_COUNT'].str.replace('-','0')
    df_label['GRADUATED_COUNT'] = df_label['GRADUATED_COUNT'].str.replace('-','0')
    # set previous change to numeric (int)
    df_label['FOUR_YR_GRAD_RATE'] = pd.to_numeric(df_label['FOUR_YR_GRAD_RATE'])
    df_label['FOUR_YR_ADJ_COHORT_COUNT'] = pd.to_numeric(df_label['FOUR_YR_ADJ_COHORT_COUNT'])
    df_label['GRADUATED_COUNT'] = pd.to_numeric(df_label['GRADUATED_COUNT'])
    grad_df.append(df_label)
    
frame = pd.concat(grad_df, axis=0, ignore_index=True)

frame = frame.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','SCHOOL_NAME','SUBGROUP','FOUR_YR_ADJ_COHORT_COUNT','FOUR_YR_GRAD_RATE','GRADUATED_COUNT'])
frame['YEAR']=frame['YEAR'].astype(int)

frame = frame.sort_values(by=['YEAR'])
frame

C:\Users\rsram\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,YEAR,COUNTY_NAME,DISTRICT_NAME,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,2014,ATLANTIC,ATLANTIC CITY,ATLANTIC CITY HIGH SCHOOL,White,113,82.30,93
5083,2014,MORRIS,MADISON BORO,Districtwide Total,Economically Disadvantaged,0,0.00,0
5082,2014,MORRIS,MADISON BORO,Districtwide Total,Limited English Proficiency,0,0.00,0
5081,2014,MORRIS,MADISON BORO,Districtwide Total,Two or More Races,0,0.00,0
5080,2014,MORRIS,MADISON BORO,Districtwide Total,Native Hawaiian,0,0.00,0
5079,2014,MORRIS,MADISON BORO,Districtwide Total,American Indian,0,0.00,0
5078,2014,MORRIS,MADISON BORO,Districtwide Total,Asian,0,0.00,0
5077,2014,MORRIS,MADISON BORO,Districtwide Total,Hispanic,0,0.00,0
5076,2014,MORRIS,MADISON BORO,Districtwide Total,Black,0,0.00,0
5075,2014,MORRIS,MADISON BORO,Districtwide Total,White,181,98.34,178


# Scrape school budget data

In [4]:
# files
rootdir = './data/district'

# lists defined
budget_list = []
years = ['2017_2018','2016_2017','2015_2016','2014_2015','2013_2014']

# for loop to get all files
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        data = os.path.join(subdir, file)
        budget_list.append(data)
        budget_list.sort()
        budget_list.reverse()
        

df_budget=[]
#  add all 4 year graduation from 2013 to 2018 to mongod
for (file, year) in zip(budget_list, years):
    data = file
#     defined variables labels
    budget_label = "district_budget"+ year
    df_label = budget_label +"_df"    
    
#     Read data & add to dataframe
    grad_label = pd.read_csv(data)
    df_label = pd.DataFrame(grad_label)
    df_label = df_label.filter(['YEAR','COUNTY_NAME', 'DISTRICT_NAME','TOTAL'])
    df_label = df_label.rename(index=str, columns={"TOTAL": "DISTRICT_BUDGET"})
#     df["a"] = pd.to_numeric(df["a"])
#     df_label["YEAR"] = pd.to_numeric(df_label["YEAR"])
    df_budget.append(df_label)

frame_budget = pd.concat(df_budget, axis=0, ignore_index=True)
frame_budget['YEAR']=frame_budget['YEAR'].astype(int)

frame_budget = frame_budget.sort_values(by=['YEAR'])

budget_df = pd.merge(frame_budget,
                frame,
                how='inner',
                on=['YEAR', 'COUNTY_NAME', 'DISTRICT_NAME'])

budget_df
#print(len(frame_budget))

,YEAR,COUNTY_NAME,DISTRICT_NAME,DISTRICT_BUDGET,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Native Hawaiian,0,0.00,0
1,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,American Indian,0,0.00,0
2,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Asian,0,0.00,0
3,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Hispanic,0,0.00,0
4,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Black,0,0.00,0
5,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,White,31,64.52,20
6,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Two or More Races,0,0.00,0
7,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Limited English Proficiency,0,0.00,0
8,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Economically Disadvantaged,39,74.36,29
9,2014,CAPE MAY,WILDWOOD CITY,5169720,WILDWOOD HIGH SCHOOL,Students with Disability,0,0.00,0


# Scrape Census data

In [5]:
census_data = './data/citylocs_geocodio.csv'
census_data = pd.read_csv(census_data)
census_df = pd.DataFrame(census_data)
census_df.columns = census_df.columns.str.replace(" ", "_")

#Drop columns not required
census_df = census_df.drop(columns=['Street', 'Country', 'State'])

#Extract all the counties to be filtered from original dataset for census
lstCounty = budget_df["COUNTY_NAME"].str.strip().tolist()
lstUniqueCounty = pd.unique(lstCounty)

#converting census data to match frame county
census_df['County'] = census_df['County'].str.upper()
census_df['County'] = census_df['County'].str.replace('COUNTY','').str.strip()

#Filter only relevant data
census_filtered_df = census_df[census_df['County'].isin(lstUniqueCounty)]

 
census_filtered_df.head()

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193
1,WOODBURY CITY,39.8382,-75.1527,Woodbury,GLOUCESTER,8096,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368
2,WOODBRIDGE TWP,40.5576,-74.2846,Woodbridge,MIDDLESEX,7095,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427
3,WOOD-RIDGE BORO,40.8457,-74.0879,Wood Ridge,BERGEN,7075,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407
4,WINSLOW TWP,39.6573,-74.8624,Winslow,CAMDEN,8095,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376


In [6]:
# Scrape population data
population_data = './data/census_pop.csv'
population_data = pd.read_csv(population_data)
population_df = pd.DataFrame(population_data)
population_df = population_df.rename(columns = {'Zipcode': 'Zip'})
population_df.head()

,Population,Zip
0,22121.0,1832
1,8295.0,1833
2,6675.0,1834
3,13527.0,1835
4,4547.0,1840


In [7]:
#Merging population with regional census data
pop_census_df = pd.merge(census_filtered_df,
                population_df,
                how='left',
                on='Zip')
pop_census_df

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,Median_age_female,"Population_by_minimum_level_of_education/Male:_12th_grade,_no_diploma",Population_by_minimum_level_of_education/Male:_High_school_graduate,Population_by_minimum_level_of_education/Female:_12th_grade_no_diploma,Population_by_minimum_level_of_education/Female:_High_school_graduate,Population
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,43.6,0.000,0.210,0.000,0.193,9129.0
1,WOODBURY CITY,39.8382,-75.1527,Woodbury,GLOUCESTER,8096,785.0,33205.0,43.0,41.7,44.3,0.015,0.450,0.000,0.368,35116.0
2,WOODBRIDGE TWP,40.5576,-74.2846,Woodbridge,MIDDLESEX,7095,462.0,82870.0,41.9,38.7,43.2,0.033,0.496,0.000,0.427,20722.0
3,WOOD-RIDGE BORO,40.8457,-74.0879,Wood Ridge,BERGEN,7075,250.0,0.0,40.0,39.9,43.0,0.000,0.356,0.000,0.407,8071.0
4,WINSLOW TWP,39.6573,-74.8624,Winslow,CAMDEN,8095,487.0,91932.0,37.8,38.8,35.1,0.000,0.384,0.039,0.376,180.0
5,WILLINGBORO TWP,40.0279,-74.8691,Willingboro,BURLINGTON,8046,703.0,76164.0,36.9,33.8,40.2,0.000,0.288,0.000,0.304,31731.0
6,WILDWOOD CITY,38.9918,-74.8149,Wildwood,CAPE MAY,8260,660.0,31020.0,45.9,47.2,45.7,0.045,0.556,0.024,0.437,13690.0
7,WESTWOOD REGIONAL,40.9912,-74.0326,Westwood,BERGEN,7675,478.0,90854.0,40.1,33.9,47.1,0.000,0.309,0.034,0.220,26540.0
8,WESTFIELD TOWN,40.6590,-74.3474,Westfield,UNION,7090,418.0,158810.0,45.3,49.4,43.6,0.000,0.073,0.000,0.104,30599.0
9,WEST ORANGE TOWN,40.7987,-74.2390,West Orange,ESSEX,7052,247.0,63164.0,51.1,44.8,56.1,0.056,0.432,0.000,0.461,46386.0


# Merge cencus and graduation data

In [8]:
popcensus_budget_df = pop_census_df.merge(budget_df, left_on=['County', 'DISTRICT_NAME'], right_on=['COUNTY_NAME', 'DISTRICT_NAME'], how='inner')
popcensus_budget_df

,DISTRICT_NAME,Latitude,Longitude,City,County,Zip,Number_of_households,Median_household_income,Median_age,Median_age_male,...,Population_by_minimum_level_of_education/Female:_High_school_graduate,Population,YEAR,COUNTY_NAME,DISTRICT_BUDGET,SCHOOL_NAME,SUBGROUP,FOUR_YR_ADJ_COHORT_COUNT,FOUR_YR_GRAD_RATE,GRADUATED_COUNT
0,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Native Hawaiian,0,0.00,0
1,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,American Indian,0,0.00,0
2,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Asian,0,0.00,0
3,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Hispanic,0,0.00,0
4,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Black,0,0.00,0
5,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,White,151,97.35,147
6,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Two or More Races,0,0.00,0
7,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Limited English Proficiency,0,0.00,0
8,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Students with Disability,0,0.00,0
9,WOODSTOWN-PILESGROVE REG,39.6515,-75.3282,Woodstown,SALEM,8098,406.0,95938.0,40.7,33.3,...,0.193,9129.0,2014,SALEM,6950561,WOODSTOWN HIGH SCHOOL,Economically Disadvantaged,0,0.00,0


In [9]:
# clean merged data set
popcensus_budget_df = popcensus_budget_df[popcensus_budget_df.GRADUATED_COUNT !=0]

# convert the dataframe to a diciontary, which can be pushed to mongo
final_dict = popcensus_budget_df.to_dict('records')

# create mongo collection
collection = db.education_data
# drop collection if duplicate
db.collection.drop()
# add data to collection
collection.insert_many(final_dict)

# df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv')
popcensus_budget_df.to_csv(r'./data/merge_data.csv')